<div align="center">
    <a href="https://colab.research.google.com/github/Kartavya-Jharwal/Kartavya_Business_Analytics2025/blob/main/A1/assignment.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
    </a>
</div>

---

# Assignment A1 - Hypothesis Testing

## Exploring the Relationship Between Economic Indicators and Global Development Outcomes

> *"The greatest threat to our planet is the belief that someone else will save it."*  
> — Robert Swan, Polar Explorer

---

<table>
<tr>
<td><strong>Course:</strong></td>
<td>Fundamentals of Business Analytics - BAN-0200</td>
</tr>
<tr>
<td><strong>Professor:</strong></td>
<td>Prof Glen Joseph</td>
</tr>
<tr>
<td><strong>Prepared by:</strong></td>
<td>Kartavya Jharwal</td>
</tr>
<tr>
<td><strong>Due Date:</strong></td>
<td>October 24, 2025</td>
</tr>
</table>

---

## Assignment Overview

This assignment explores the relationship between economic prosperity and environmental/social outcomes by examining:

1. **GDP per capita**
2. **CO₂ emissions per capita**
3. **Net-zero carbon emissions targets**

> ### Core Hypothesis (Part 1):
>
> *"Countries with higher GDP per capita emit more CO₂ per capita."*

### Objectives

1. **Part 1:** Test the core hypothesis using provided GDP and CO₂ datasets
2. **Part 2:** Extend analysis with net-zero carbon emissions targets and new hypothesis
3. Apply statistical methods including confidence intervals and descriptive analytics
4. Create compelling visualizations to support findings
5. Provide critical interpretation of results with contextual understanding

---


In [1]:
# Import necessary libraries for data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import (shapiro, skew, kurtosis, pearsonr, spearmanr,
                         f_oneway, ttest_ind, chi2_contingency)
from itertools import combinations
import warnings
import sys
import platform
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plotting style and parameters
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Environment and system information
print("ASSIGNMENT A1 - BUSINESS ANALYTICS")
print("="*60)
print("Execution Date: " + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print("Python Version: " + sys.version)
print("Platform: " + platform.platform())
print("Architecture: " + platform.architecture()[0])

print("\n" + "="*60)
print("LIBRARY VERSIONS")
print("="*60)
print("✓ Pandas: " + pd.__version__)
print("✓ NumPy: " + np.__version__)
print("✓ Matplotlib: " + plt.matplotlib.__version__)
print("✓ Seaborn: " + sns.__version__)
print("✓ SciPy: " + (stats.__version__ if hasattr(stats, '__version__') else 'Available'))

# Check if running in Google Colab
try:
    import google.colab
    print("✓ Google Colab: Detected")
    colab_env = True
except ImportError:
    print("✓ Environment: Local/Other")
    colab_env = False

print("="*60)

ASSIGNMENT A1 - BUSINESS ANALYTICS
Execution Date: 2025-10-04 13:31:01
Python Version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Platform: Linux-6.6.97+-x86_64-with-glibc2.35
Architecture: 64bit

LIBRARY VERSIONS
✓ Pandas: 2.2.2
✓ NumPy: 2.0.2
✓ Matplotlib: 3.10.0
✓ Seaborn: 0.13.2
✓ SciPy: Available
✓ Google Colab: Detected


# Part 1: Hypothesis Testing with Provided Datasets

## Core Hypothesis

> *"Countries with higher GDP per capita emit more CO₂ per capita."*

### Datasets to be Analyzed

#### 1. CO₂ Emissions per Capita

```
co-emissions-per-capita/co-emissions-per-capita.csv
```

**Source:** Global Carbon Budget (2024), Population based on various sources (2024) – with major processing by Our World in Data

#### 2. GDP per Capita in Constant USD

```
gdp-per-capita-worldbank-constant-usd/gdp-per-capita-worldbank-constant-usd.csv
```

**Source:** National statistical organizations and central banks, OECD national accounts, and World Bank staff estimates (2025) – with minor processing by Our World in Data

### Analysis Steps

1. Load and inspect both datasets
2. Clean and standardize the data
3. Merge datasets on Country and Year
4. Create GDP categories (Low, Medium, High)
5. Calculate descriptive statistics with confidence intervals
6. Create visualizations
7. Interpret results

---

## Step 1: Load and Inspect Datasets

In [2]:
# GitHub base URL for datasets
github_base = "https://raw.githubusercontent.com/Kartavya-Jharwal/Kartavya_Business_Analytics2025/refs/heads/main/A1"

# Define dataset URLs
co2_url = github_base + "/co-emissions-per-capita/co-emissions-per-capita.csv"
gdp_url = github_base + "/gdp-per-capita-worldbank-constant-usd/gdp-per-capita-worldbank-constant-usd.csv"

print("="*60)
print("LOADING DATASETS")
print("="*60)

# Load CO2 emissions dataset
print("\n1. Loading CO2 emissions dataset...")
co2_df = pd.read_csv(co2_url)
print(f"   ✓ CO2 dataset loaded: {co2_df.shape[0]} rows, {co2_df.shape[1]} columns")

# Load GDP dataset
print("\n2. Loading GDP dataset...")
gdp_df = pd.read_csv(gdp_url)
print(f"   ✓ GDP dataset loaded: {gdp_df.shape[0]} rows, {gdp_df.shape[1]} columns")

print("\n" + "="*60)
print("DATA LOADING COMPLETE")
print("="*60)

LOADING DATASETS

1. Loading CO2 emissions dataset...
   ✓ CO2 dataset loaded: 26317 rows, 4 columns

2. Loading GDP dataset...
   ✓ GDP dataset loaded: 12098 rows, 4 columns

DATA LOADING COMPLETE


In [3]:
# Inspect CO2 dataset
print("="*60)
print("CO2 EMISSIONS DATASET")
print("="*60)

print("\nFirst 5 rows:")
display(co2_df.head())

print("\nColumn names:")
print(co2_df.columns.tolist())

print("\nDataset shape:", co2_df.shape)
print("Year range:", co2_df['Year'].min(), "-", co2_df['Year'].max())

print("\nMissing values:")
print(co2_df.isnull().sum())

# Inspect GDP dataset
print("\n" + "="*60)
print("GDP DATASET")
print("="*60)

print("\nFirst 5 rows:")
display(gdp_df.head())

print("\nColumn names:")
print(gdp_df.columns.tolist())

print("\nDataset shape:", gdp_df.shape)
print("Year range:", gdp_df['Year'].min(), "-", gdp_df['Year'].max())

print("\nMissing values:")
print(gdp_df.isnull().sum())

CO2 EMISSIONS DATASET

First 5 rows:


,Entity,Code,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,AFG,1949,0.001992
1,Afghanistan,AFG,1950,0.010837
2,Afghanistan,AFG,1951,0.011625
3,Afghanistan,AFG,1952,0.011468
4,Afghanistan,AFG,1953,0.013123



Column names:
['Entity', 'Code', 'Year', 'Annual CO₂ emissions (per capita)']

Dataset shape: (26317, 4)
Year range: 1750 - 2023

Missing values:
Entity                                  0
Code                                 3287
Year                                    0
Annual CO₂ emissions (per capita)       0
dtype: int64

GDP DATASET

First 5 rows:


,Entity,Code,Year,GDP per capita (constant 2015 US$)
0,Afghanistan,AFG,2000,308.31827
1,Afghanistan,AFG,2001,277.11804
2,Afghanistan,AFG,2002,338.13998
3,Afghanistan,AFG,2003,346.07162
4,Afghanistan,AFG,2004,338.63727



Column names:
['Entity', 'Code', 'Year', 'GDP per capita (constant 2015 US$)']

Dataset shape: (12098, 4)
Year range: 1960 - 2024

Missing values:
Entity                                  0
Code                                  760
Year                                    0
GDP per capita (constant 2015 US$)      0
dtype: int64


## Step 2: Clean and Standardize Data

Before merging the datasets, we need to:

1. **Standardize country names** between datasets
2. **Identify overlapping years** across both datasets
3. **Handle missing or inconsistent data points**
4. **Ensure data quality** for meaningful analysis

In [4]:
# Clean CO2 dataset - Make a copy first
co2_clean = co2_df.copy()

print("="*60)
print("CLEANING CO2 DATASET")
print("="*60)

# Check initial size
print(f"Initial rows: {len(co2_clean)}")

# Remove rows with missing Entity or Year
co2_clean = co2_clean.dropna(subset=['Entity', 'Year'])
print(f"After removing missing Entity/Year: {len(co2_clean)} rows")

# Check unique countries and years
print(f"Unique countries: {co2_clean['Entity'].nunique()}")
print(f"Year range: {co2_clean['Year'].min()} - {co2_clean['Year'].max()}")

# Clean GDP dataset - Make a copy first
gdp_clean = gdp_df.copy()

print("\n" + "="*60)
print("CLEANING GDP DATASET")
print("="*60)

# Check initial size
print(f"Initial rows: {len(gdp_clean)}")

# Remove rows with missing Entity or Year
gdp_clean = gdp_clean.dropna(subset=['Entity', 'Year'])
print(f"After removing missing Entity/Year: {len(gdp_clean)} rows")

# Check unique countries and years
print(f"Unique countries: {gdp_clean['Entity'].nunique()}")
print(f"Year range: {gdp_clean['Year'].min()} - {gdp_clean['Year'].max()}")

# Check for common countries
co2_countries = set(co2_clean['Entity'].unique())
gdp_countries = set(gdp_clean['Entity'].unique())
common_countries = co2_countries.intersection(gdp_countries)

print("\n" + "="*60)
print("OVERLAP ANALYSIS")
print("="*60)
print(f"Common countries: {len(common_countries)}")
print(f"Countries only in CO2: {len(co2_countries - gdp_countries)}")
print(f"Countries only in GDP: {len(gdp_countries - co2_countries)}")

CLEANING CO2 DATASET
Initial rows: 26317
After removing missing Entity/Year: 26317 rows
Unique countries: 231
Year range: 1750 - 2023

CLEANING GDP DATASET
Initial rows: 12098
After removing missing Entity/Year: 12098 rows
Unique countries: 225
Year range: 1960 - 2024

OVERLAP ANALYSIS
Common countries: 208
Countries only in CO2: 23
Countries only in GDP: 17


## Step 3: Merge Datasets

**Data Integration Process**

We'll merge the cleaned CO₂ and GDP datasets on Country and Year to create our analysis dataset. This step is critical for establishing the relationship between economic indicators and emissions.

**Key Operations:**

- Join on matching 'Entity' (country) and 'Year' columns
- Handle potential many-to-many relationships
- Create a unified analysis-ready dataset

In [5]:
# Merge the two datasets on Country (Entity) and Year
print("="*60)
print("MERGING DATASETS")
print("="*60)

# Rename Entity to Country for clarity
co2_merge = co2_clean.copy()
gdp_merge = gdp_clean.copy()

# Rename columns
co2_merge = co2_merge.rename(columns={'Entity': 'Country'})
gdp_merge = gdp_merge.rename(columns={'Entity': 'Country'})

print(f"CO2 dataset: {len(co2_merge)} rows")
print(f"GDP dataset: {len(gdp_merge)} rows")

# Perform inner merge (only keep matching records)
merged_data = pd.merge(
    co2_merge,
    gdp_merge,
    on=['Country', 'Year'],
    how='inner',
    suffixes=('_co2', '_gdp')
)

print(f"\nMerged dataset: {len(merged_data)} rows")
print(f"Countries in merged data: {merged_data['Country'].nunique()}")
print(f"Year range: {merged_data['Year'].min()} - {merged_data['Year'].max()}")

print("\nColumn names in merged data:")
print(merged_data.columns.tolist())

print("\nFirst 5 rows of merged data:")
display(merged_data.head())

MERGING DATASETS
CO2 dataset: 26317 rows
GDP dataset: 12098 rows

Merged dataset: 11001 rows
Countries in merged data: 208
Year range: 1960 - 2023

Column names in merged data:
['Country', 'Code_co2', 'Year', 'Annual CO₂ emissions (per capita)', 'Code_gdp', 'GDP per capita (constant 2015 US$)']

First 5 rows of merged data:


,Country,Code_co2,Year,Annual CO₂ emissions (per capita),Code_gdp,GDP per capita (constant 2015 US$)
0,Afghanistan,AFG,2000,0.052018,AFG,308.31827
1,Afghanistan,AFG,2001,0.052706,AFG,277.11804
2,Afghanistan,AFG,2002,0.062728,AFG,338.13998
3,Afghanistan,AFG,2003,0.068605,AFG,346.07162
4,Afghanistan,AFG,2004,0.052513,AFG,338.63727


## Step 4: Feature Engineering - GDP Categories

Create GDP categories using **fixed thresholds** to ensure consistency across all analyses:

- **Low GDP:** < $5,000 per capita

- **Medium GDP:** \$5,000 - \$15,000 per capita

- **High GDP:** > $15,000 per capita

**Note:** These categories are for descriptive analysis only. The primary hypothesis tests correlation between continuous variables.

In [6]:
# Create GDP categories using FIXED thresholds for consistency
print("="*60)
print("CREATING GDP CATEGORIES")
print("="*60)

# Make a copy for analysis
analysis_df = merged_data.copy()

# Find the GDP column
gdp_columns = [col for col in analysis_df.columns if 'gdp' in col.lower() and 'capita' in col.lower()]
print(f"GDP columns found: {gdp_columns}")

# Select the GDP column (first match)
gdp_col = gdp_columns[0]
print(f"Using GDP column: '{gdp_col}'")

# Convert to numeric if needed
analysis_df[gdp_col] = pd.to_numeric(analysis_df[gdp_col], errors='coerce')

# Remove any missing GDP values
analysis_df = analysis_df.dropna(subset=[gdp_col])
print(f"Rows after removing missing GDP: {len(analysis_df)}")

# FIXED THRESHOLDS (consistent across all analyses)
threshold_low = 5000
threshold_high = 15000

print("\nFixed Thresholds (ensures everyone gets same categories):")
print(f"  Low GDP:    < ${threshold_low:,}")
print(f"  Medium GDP: ${threshold_low:,} - ${threshold_high:,}")
print(f"  High GDP:   > ${threshold_high:,}")

# Create GDP categories based on FIXED thresholds
analysis_df['GDP_Category'] = pd.cut(
    analysis_df[gdp_col],
    bins=[-np.inf, threshold_low, threshold_high, np.inf],
    labels=['Low', 'Medium', 'High']
)

# Show distribution
print("\nGDP Category Distribution:")
category_counts = analysis_df['GDP_Category'].value_counts()
total = len(analysis_df)
for category in ['Low', 'Medium', 'High']:
    if category in category_counts.index:
        count = category_counts[category]
        percentage = (count / total) * 100
        print(f"  {category}: {count} observations ({percentage:.1f}%)")

# Show GDP statistics by category
print("\nGDP Statistics by Category:")
gdp_stats = analysis_df.groupby('GDP_Category')[gdp_col].agg(['count', 'mean', 'median', 'std', 'min', 'max']).round(2)
display(gdp_stats)

print("\n" + "="*60)
print("NOTE: Categories use FIXED thresholds for consistency.")
print("Core hypothesis tests correlation between continuous variables.")
print("="*60)

CREATING GDP CATEGORIES
GDP columns found: ['GDP per capita (constant 2015 US$)']
Using GDP column: 'GDP per capita (constant 2015 US$)'
Rows after removing missing GDP: 11001

Fixed Thresholds (ensures everyone gets same categories):
  Low GDP:    < $5,000
  Medium GDP: $5,000 - $15,000
  High GDP:   > $15,000

GDP Category Distribution:
  Low: 6178 observations (56.2%)
  Medium: 2120 observations (19.3%)
  High: 2703 observations (24.6%)

GDP Statistics by Category:


,count,mean,median,std,min,max
GDP_Category,,,,,,
Low,6178,1871.48,1499.11,1299.81,122.68,4998.67
Medium,2120,8659.76,8106.04,2795.63,5000.99,14993.69
High,2703,35020.49,29547.76,19418.11,15003.44,167187.16



NOTE: Categories use FIXED thresholds for consistency.
Core hypothesis tests correlation between continuous variables.


## Statistical Hypothesis Formulation

Before conducting the analysis, we must formally state our hypotheses:

### Null Hypothesis (H₀)

**Statement:** There is no linear relationship between GDP per capita and CO₂ emissions per capita.

**Mathematical Notation:**

$$H_0: \rho = 0$$

Where ρ (rho) is the population correlation coefficient between GDP per capita and CO₂ emissions per capita.

### Alternative Hypothesis (H₁)

**Statement:** There is a positive linear relationship between GDP per capita and CO₂ emissions per capita. As GDP per capita increases, CO₂ emissions per capita also increase.

**Mathematical Notation:**

$$H_1: \rho > 0$$

### Significance Level

α = 0.05 (5% significance level)

**Decision Rule:**
- If p-value < 0.05, reject H₀ (evidence of significant positive correlation)
- If p-value ≥ 0.05, fail to reject H₀ (insufficient evidence of correlation)

**Note:** GDP categories (Low, Medium, High) are created for descriptive analysis and visualization purposes only. The core hypothesis tests the relationship between the continuous variables.

## Distribution Analysis: Checking Assumptions

Before applying parametric tests (correlation), we must verify that our **continuous variables** meet necessary assumptions:

1. **Normality** - Are GDP and CO₂ normally distributed?
2. **Skewness** - Is the distribution symmetrical or skewed?
3. **Kurtosis** - Does the distribution have heavy tails or outliers?
4. **Linearity** - Is the relationship between variables linear?

These checks determine whether we can use Pearson correlation or need Spearman correlation (non-parametric alternative).

In [7]:
# Normality Test for Continuous Variables

print("="*80)
print("NORMALITY TESTING: SHAPIRO-WILK TEST")
print("="*80)

# Get continuous variables
gdp_col = [col for col in analysis_df.columns if 'gdp' in col.lower() and 'capita' in col.lower()][0]
co2_col = [col for col in analysis_df.columns if 'co2' in col.lower() or 'emission' in col.lower()]
co2_col = [c for c in co2_col if 'code' not in c.lower()][0]

# Clean data
clean_data = analysis_df[[gdp_col, co2_col]].dropna()

print("\nWhat is Shapiro-Wilk Test?")
print("  • Tests if data follows a normal (Gaussian) distribution")
print("  • H₀: Data is normally distributed")
print("  • H₁: Data is NOT normally distributed")
print("  • If p < 0.05: Reject H₀ (data not normal)")

print("\n" + "-"*80)
print("RESULTS")
print("-"*80)

# Test GDP per capita
print(f"\n1. GDP per Capita (n={len(clean_data)}):")
# For large samples, use a sample for Shapiro-Wilk (max 5000)
if len(clean_data) > 5000:
    gdp_sample = clean_data[gdp_col].sample(5000, random_state=42)
    print(f"   (Using random sample of 5000 for computational efficiency)")
else:
    gdp_sample = clean_data[gdp_col]

stat_gdp, p_gdp = shapiro(gdp_sample)
print(f"   Statistic: {stat_gdp:.6f}")
print(f"   P-value: {p_gdp:.6f}")
print(f"   Conclusion: {'NOT normal' if p_gdp < 0.05 else 'Approximately normal'} (α=0.05)")

# Test CO2 emissions
print(f"\n2. CO₂ Emissions per Capita (n={len(clean_data)}):")
if len(clean_data) > 5000:
    co2_sample = clean_data[co2_col].sample(5000, random_state=42)
    print(f"   (Using random sample of 5000 for computational efficiency)")
else:
    co2_sample = clean_data[co2_col]

stat_co2, p_co2 = shapiro(co2_sample)
print(f"   Statistic: {stat_co2:.6f}")
print(f"   P-value: {p_co2:.6f}")
print(f"   Conclusion: {'NOT normal' if p_co2 < 0.05 else 'Approximately normal'} (α=0.05)")

print("\n" + "="*80)
print("INTERPRETATION")
print("="*80)

if p_gdp < 0.05 or p_co2 < 0.05:
    print("⚠ At least one variable is NOT normally distributed")
    print("\nRecommendations:")
    print("  • Use BOTH Pearson and Spearman correlation")
    print("  • Spearman is more robust to non-normality")
    print("  • Large sample size (n > 1000) → Central Limit Theorem applies")
    print("  • Results should be similar if relationship is monotonic")
else:
    print("✓ Both variables are approximately normally distributed")
    print("  • Pearson correlation is appropriate")
    print("  • Can also use Spearman for confirmation")

print("\nNote: With large samples (n > 1000), parametric tests are robust to")
print("moderate departures from normality due to the Central Limit Theorem.")
print("="*80)

NORMALITY TESTING: SHAPIRO-WILK TEST

What is Shapiro-Wilk Test?
  • Tests if data follows a normal (Gaussian) distribution
  • H₀: Data is normally distributed
  • H₁: Data is NOT normally distributed
  • If p < 0.05: Reject H₀ (data not normal)

--------------------------------------------------------------------------------
RESULTS
--------------------------------------------------------------------------------

1. GDP per Capita (n=11001):
   (Using random sample of 5000 for computational efficiency)
   Statistic: 0.652950
   P-value: 0.000000
   Conclusion: NOT normal (α=0.05)

2. CO₂ Emissions per Capita (n=11001):
   (Using random sample of 5000 for computational efficiency)
   Statistic: 0.627702
   P-value: 0.000000
   Conclusion: NOT normal (α=0.05)

INTERPRETATION
⚠ At least one variable is NOT normally distributed

Recommendations:
  • Use BOTH Pearson and Spearman correlation
  • Spearman is more robust to non-normality
  • Large sample size (n > 1000) → Central Limit Theo

## Skewness and Kurtosis Analysis

Examine the shape of both continuous variables to understand asymmetry and tail behavior.

In [8]:
# Skewness and Kurtosis Analysis for Continuous Variables

print("="*80)
print("DISTRIBUTION SHAPE ANALYSIS: SKEWNESS AND KURTOSIS")
print("="*80)

# Get continuous variables
gdp_col = [col for col in analysis_df.columns if 'gdp' in col.lower() and 'capita' in col.lower()][0]
co2_col = [col for col in analysis_df.columns if 'co2' in col.lower() or 'emission' in col.lower()]
co2_col = [c for c in co2_col if 'code' not in c.lower()][0]

# Clean data
clean_data = analysis_df[[gdp_col, co2_col]].dropna()

print("\nWhat do these metrics tell us?\n")
print("SKEWNESS:")
print("  • Measures asymmetry of the distribution")
print("  • 0 = perfectly symmetrical (normal distribution)")
print("  • Positive (+) = right tail (most values on left, few high outliers)")
print("  • Negative (-) = left tail (most values on right, few low outliers)")
print("  • |skewness| < 0.5: approximately symmetric")
print("  • |skewness| 0.5-1: moderately skewed")
print("  • |skewness| > 1: highly skewed")

print("\nKURTOSIS (Excess Kurtosis):")
print("  • Measures tail heaviness / outlier presence")
print("  • 0 = normal distribution tails (mesokurtic)")
print("  • Positive (+) = heavy tails, more outliers (leptokurtic)")
print("  • Negative (-) = light tails, fewer outliers (platykurtic)")
print("  • |kurtosis| < 1: approximately normal tails")
print("  • |kurtosis| > 3: significant deviation from normal\n")

print("="*80)
print("RESULTS FOR CONTINUOUS VARIABLES")
print("="*80)

# Analyze GDP per capita
gdp_data = clean_data[gdp_col]
gdp_skewness = skew(gdp_data)
gdp_kurtosis = kurtosis(gdp_data)

print(f"\n1. GDP per Capita (n={len(gdp_data)}):")
print(f"   Mean: ${gdp_data.mean():,.2f}")
print(f"   Median: ${gdp_data.median():,.2f}")
print(f"   Std Dev: ${gdp_data.std():,.2f}")
print(f"   Min: ${gdp_data.min():,.2f}")
print(f"   Max: ${gdp_data.max():,.2f}")
print(f"   ---")
print(f"   Skewness: {gdp_skewness:.4f}", end="")

# Interpret skewness
if abs(gdp_skewness) < 0.5:
    skew_interp = "approximately symmetric"
elif abs(gdp_skewness) < 1:
    skew_interp = f"moderately {'right' if gdp_skewness > 0 else 'left'}-skewed"
else:
    skew_interp = f"highly {'right' if gdp_skewness > 0 else 'left'}-skewed"
print(f" ({skew_interp})")

print(f"   Kurtosis: {gdp_kurtosis:.4f}", end="")

# Interpret kurtosis
if abs(gdp_kurtosis) < 0.5:
    kurt_interp = "normal tails"
elif gdp_kurtosis > 3:
    kurt_interp = "very heavy tails, many outliers"
elif gdp_kurtosis > 0:
    kurt_interp = "heavy tails, some outliers"
else:
    kurt_interp = "light tails, few outliers"
print(f" ({kurt_interp})")

# Analyze CO2 emissions
co2_data = clean_data[co2_col]
co2_skewness = skew(co2_data)
co2_kurtosis = kurtosis(co2_data)

print(f"\n2. CO₂ Emissions per Capita (n={len(co2_data)}):")
print(f"   Mean: {co2_data.mean():.4f} tonnes")
print(f"   Median: {co2_data.median():.4f} tonnes")
print(f"   Std Dev: {co2_data.std():.4f} tonnes")
print(f"   Min: {co2_data.min():.4f} tonnes")
print(f"   Max: {co2_data.max():.4f} tonnes")
print(f"   ---")
print(f"   Skewness: {co2_skewness:.4f}", end="")

# Interpret skewness
if abs(co2_skewness) < 0.5:
    skew_interp = "approximately symmetric"
elif abs(co2_skewness) < 1:
    skew_interp = f"moderately {'right' if co2_skewness > 0 else 'left'}-skewed"
else:
    skew_interp = f"highly {'right' if co2_skewness > 0 else 'left'}-skewed"
print(f" ({skew_interp})")

print(f"   Kurtosis: {co2_kurtosis:.4f}", end="")

# Interpret kurtosis
if abs(co2_kurtosis) < 0.5:
    kurt_interp = "normal tails"
elif co2_kurtosis > 3:
    kurt_interp = "very heavy tails, many outliers"
elif co2_kurtosis > 0:
    kurt_interp = "heavy tails, some outliers"
else:
    kurt_interp = "light tails, few outliers"
print(f" ({kurt_interp})")

# Summary table
print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)

summary_data = pd.DataFrame({
    'Variable': ['GDP per Capita', 'CO₂ Emissions'],
    'n': [len(gdp_data), len(co2_data)],
    'Mean': [gdp_data.mean(), co2_data.mean()],
    'Median': [gdp_data.median(), co2_data.median()],
    'Std_Dev': [gdp_data.std(), co2_data.std()],
    'Skewness': [gdp_skewness, co2_skewness],
    'Kurtosis': [gdp_kurtosis, co2_kurtosis]
})
display(summary_data.round(4))

# Overall interpretation
print("\n" + "="*80)
print("INTERPRETATION FOR CORRELATION ANALYSIS")
print("="*80)

# Check if any variable is problematic
problematic_skew = any(abs(summary_data['Skewness']) > 1)
problematic_kurt = any(abs(summary_data['Kurtosis']) > 3)

if problematic_skew or problematic_kurt:
    print("⚠ CAUTION: Data shows significant departures from normality")
    print("\nRecommendations:")
    if problematic_skew:
        print("  • High skewness detected → Data is asymmetric")
        print("  • Spearman correlation preferred (robust to skewness)")
        print("  • Consider log transformation for visualization")
    if problematic_kurt:
        print("  • High kurtosis detected → Outliers present")
        print("  • Spearman correlation more robust to outliers")
        print("  • Pearson correlation can be influenced by extreme values")
    print("\n✓ We will use BOTH Pearson and Spearman correlation")
    print("  • Pearson: Tests linear relationship")
    print("  • Spearman: Tests monotonic relationship (more robust)")
else:
    print("✓ Data is reasonably close to normal distribution")
    print("  • Pearson correlation is appropriate")
    print("  • Spearman correlation will provide confirmation")

print("\nNote: Large sample size (n > 1000) provides robustness via Central Limit Theorem")
print("="*80)

DISTRIBUTION SHAPE ANALYSIS: SKEWNESS AND KURTOSIS

What do these metrics tell us?

SKEWNESS:
  • Measures asymmetry of the distribution
  • 0 = perfectly symmetrical (normal distribution)
  • Positive (+) = right tail (most values on left, few high outliers)
  • Negative (-) = left tail (most values on right, few low outliers)
  • |skewness| < 0.5: approximately symmetric
  • |skewness| 0.5-1: moderately skewed
  • |skewness| > 1: highly skewed

KURTOSIS (Excess Kurtosis):
  • Measures tail heaviness / outlier presence
  • 0 = normal distribution tails (mesokurtic)
  • Positive (+) = heavy tails, more outliers (leptokurtic)
  • Negative (-) = light tails, fewer outliers (platykurtic)
  • |kurtosis| < 1: approximately normal tails
  • |kurtosis| > 3: significant deviation from normal

RESULTS FOR CONTINUOUS VARIABLES

1. GDP per Capita (n=11001):
   Mean: $11,324.53
   Median: $3,874.33
   Std Dev: $16,870.29
   Min: $122.68
   Max: $167,187.16
   ---
   Skewness: 2.6644 (highly right-

,Variable,n,Mean,Median,Std_Dev,Skewness,Kurtosis
0,GDP per Capita,11001,11324.5260,3874.3271,16870.2891,2.6644,9.1673
1,CO₂ Emissions,11001,4.6092,1.9522,7.6786,11.9710,451.7347



INTERPRETATION FOR CORRELATION ANALYSIS
⚠ CAUTION: Data shows significant departures from normality

Recommendations:
  • High skewness detected → Data is asymmetric
  • Spearman correlation preferred (robust to skewness)
  • Consider log transformation for visualization
  • High kurtosis detected → Outliers present
  • Spearman correlation more robust to outliers
  • Pearson correlation can be influenced by extreme values

✓ We will use BOTH Pearson and Spearman correlation
  • Pearson: Tests linear relationship
  • Spearman: Tests monotonic relationship (more robust)

Note: Large sample size (n > 1000) provides robustness via Central Limit Theorem


## Visual Distribution Analysis

Create histograms and Q-Q plots to visually assess normality and distribution shape for each GDP category.

In [ ]:
# Visual Distribution Analysis: Histograms and Q-Q Plots
from scipy import stats

# Get continuous variables
gdp_col = [col for col in analysis_df.columns if 'gdp' in col.lower() and 'capita' in col.lower()][0]
co2_col = [col for col in analysis_df.columns if 'co2' in col.lower() or 'emission' in col.lower()]
co2_col = [c for c in co2_col if 'code' not in c.lower()][0]

# Clean data
clean_data = analysis_df[[gdp_col, co2_col]].dropna()

# Create figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Distribution Analysis: Histograms and Q-Q Plots for Continuous Variables',
             fontsize=16, fontweight='bold', y=0.995)

# GDP per Capita - Histogram
ax1 = axes[0, 0]
gdp_data = clean_data[gdp_col]
ax1.hist(gdp_data, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(gdp_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ${gdp_data.mean():,.0f}')
ax1.axvline(gdp_data.median(), color='green', linestyle='--', linewidth=2, label=f'Median: ${gdp_data.median():,.0f}')
ax1.set_xlabel('GDP per Capita (USD)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax1.set_title('GDP per Capita Distribution', fontsize=12, fontweight='bold', pad=10)
ax1.legend(loc='upper right', fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.text(0.02, 0.98, f'n = {len(gdp_data)}\nSkewness: {skew(gdp_data):.3f}\nKurtosis: {kurtosis(gdp_data):.3f}',
         transform=ax1.transAxes, fontsize=9, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# GDP per Capita - Q-Q Plot
ax2 = axes[0, 1]
stats.probplot(gdp_data, dist="norm", plot=ax2)
ax2.set_title('GDP per Capita Q-Q Plot', fontsize=12, fontweight='bold', pad=10)
ax2.set_xlabel('Theoretical Quantiles', fontsize=11, fontweight='bold')
ax2.set_ylabel('Sample Quantiles', fontsize=11, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.get_lines()[0].set_markerfacecolor('steelblue')
ax2.get_lines()[0].set_markeredgecolor('black')
ax2.get_lines()[0].set_markersize(4)
ax2.text(0.02, 0.98, 'Points on red line = normal distribution\nDeviations indicate non-normality',
         transform=ax2.transAxes, fontsize=9, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

# CO2 Emissions - Histogram
ax3 = axes[1, 0]
co2_data = clean_data[co2_col]
ax3.hist(co2_data, bins=50, edgecolor='black', alpha=0.7, color='coral')
ax3.axvline(co2_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {co2_data.mean():.2f} tonnes')
ax3.axvline(co2_data.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {co2_data.median():.2f} tonnes')
ax3.set_xlabel('CO₂ Emissions per Capita (tonnes)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax3.set_title('CO₂ Emissions Distribution', fontsize=12, fontweight='bold', pad=10)
ax3.legend(loc='upper right', fontsize=9)
ax3.grid(True, alpha=0.3)
ax3.text(0.02, 0.98, f'n = {len(co2_data)}\nSkewness: {skew(co2_data):.3f}\nKurtosis: {kurtosis(co2_data):.3f}',
         transform=ax3.transAxes, fontsize=9, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# CO2 Emissions - Q-Q Plot
ax4 = axes[1, 1]
stats.probplot(co2_data, dist="norm", plot=ax4)
ax4.set_title('CO₂ Emissions Q-Q Plot', fontsize=12, fontweight='bold', pad=10)
ax4.set_xlabel('Theoretical Quantiles', fontsize=11, fontweight='bold')
ax4.set_ylabel('Sample Quantiles', fontsize=11, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.get_lines()[0].set_markerfacecolor('coral')
ax4.get_lines()[0].set_markeredgecolor('black')
ax4.get_lines()[0].set_markersize(4)
ax4.text(0.02, 0.98, 'Points on red line = normal distribution\nDeviations indicate non-normality',
         transform=ax4.transAxes, fontsize=9, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("INTERPRETING THE PLOTS")
print("="*80)
print("\nHISTOGRAMS:")
print("  • Show the shape and distribution of actual data")
print("  • Symmetric shape suggests normality")
print("  • Mean ≈ Median suggests symmetric distribution")
print("  • Long tails indicate outliers")

print("\nQ-Q PLOTS (Quantile-Quantile):")
print("  • Compare data quantiles to theoretical normal distribution quantiles")
print("  • Points close to red line = data is approximately normal")
print("  • Deviations from line indicate departures from normality:")
print("    - S-curve = skewed distribution")
print("    - Points above line at ends = heavy tails (high kurtosis)")
print("    - Points below line at ends = light tails (low kurtosis)")

print("\nIMPLICATIONS FOR CORRELATION ANALYSIS:")
print("  • If both variables approximately normal → Pearson correlation valid")
print("  • If significant departures from normality → Spearman more appropriate")
print("  • Our analysis includes BOTH Pearson and Spearman for robustness")
print("="*80)

## ⚠️ Note on Linearity Assumption

For correlation analysis between continuous variables, the key assumption is **linearity** of the relationship. We should visually inspect whether GDP and CO₂ have a linear or monotonic relationship using a scatterplot.

In [ ]:
# Linearity Check: Scatterplot with Regression Line

print("="*80)
print("LINEARITY ASSESSMENT: VISUAL INSPECTION OF RELATIONSHIP")
print("="*80)

# Get continuous variables
gdp_col = [col for col in analysis_df.columns if 'gdp' in col.lower() and 'capita' in col.lower()][0]
co2_col = [col for col in analysis_df.columns if 'co2' in col.lower() or 'emission' in col.lower()]
co2_col = [c for c in co2_col if 'code' not in c.lower()][0]

# Clean data
clean_data = analysis_df[[gdp_col, co2_col]].dropna()
gdp_data = clean_data[gdp_col]
co2_data = clean_data[co2_col]

print("\nWhy check for linearity?")
print("  • Pearson correlation measures LINEAR relationships")
print("  • If relationship is non-linear but monotonic → Spearman more appropriate")
print("  • If relationship is non-monotonic → correlation not meaningful")

# Create scatterplot with regression line
fig, axes = plt.subplots(1, 2, figsize=(18, 7))
fig.suptitle('Linearity Assessment: GDP vs CO₂ Emissions', fontsize=16, fontweight='bold')

# Left plot: Standard scale
ax1 = axes[0]
ax1.scatter(gdp_data, co2_data, alpha=0.5, s=30, edgecolors='black', linewidth=0.5)

# Fit linear regression line
z = np.polyfit(gdp_data, co2_data, 1)
p = np.poly1d(z)
x_line = np.linspace(gdp_data.min(), gdp_data.max(), 100)
ax1.plot(x_line, p(x_line), "r--", linewidth=2, label=f'Linear Fit: y = {z[0]:.6f}x + {z[1]:.4f}')

# Calculate Pearson correlation
r, _ = pearsonr(gdp_data, co2_data)
ax1.set_xlabel('GDP per Capita (USD)', fontsize=12, fontweight='bold')
ax1.set_ylabel('CO₂ Emissions per Capita (tonnes)', fontsize=12, fontweight='bold')
ax1.set_title('Standard Scale', fontsize=13, fontweight='bold', pad=10)
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.text(0.98, 0.02, f'Pearson r = {r:.4f}\nn = {len(clean_data)}',
         transform=ax1.transAxes, fontsize=11, verticalalignment='bottom', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

# Right plot: Log scale for better visualization of full range
ax2 = axes[1]
# Filter out zeros for log scale
clean_positive = clean_data[(clean_data[gdp_col] > 0) & (clean_data[co2_col] > 0)]
gdp_positive = clean_positive[gdp_col]
co2_positive = clean_positive[co2_col]

ax2.scatter(gdp_positive, co2_positive, alpha=0.5, s=30, edgecolors='black', linewidth=0.5, color='coral')

# Fit on log scale
z_log = np.polyfit(np.log10(gdp_positive), np.log10(co2_positive), 1)
p_log = np.poly1d(z_log)
x_log_line = np.logspace(np.log10(gdp_positive.min()), np.log10(gdp_positive.max()), 100)
ax2.plot(x_log_line, 10**(p_log(np.log10(x_log_line))), "r--", linewidth=2,
         label=f'Power Law Fit: y = {10**z_log[1]:.4f} × x^{z_log[0]:.4f}')

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('GDP per Capita (USD, log scale)', fontsize=12, fontweight='bold')
ax2.set_ylabel('CO₂ Emissions per Capita (tonnes, log scale)', fontsize=12, fontweight='bold')
ax2.set_title('Log-Log Scale (for full range visualization)', fontsize=13, fontweight='bold', pad=10)
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3, which='both')
ax2.text(0.98, 0.02, f'n = {len(clean_positive)}\n(positive values only)',
         transform=ax2.transAxes, fontsize=11, verticalalignment='bottom', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("INTERPRETATION")
print("="*80)

print("\nWhat to look for:")
print("  • LINEAR pattern → Data points form a straight line")
print("    → Pearson correlation appropriate")
print("  • MONOTONIC pattern → Consistent increase but not straight")
print("    → Spearman correlation more appropriate")
print("  • CURVED pattern → Non-linear relationship")
print("    → May need transformation or non-linear methods")
print("  • NO PATTERN → No correlation")
print("    → Neither method will find meaningful relationship")

print("\nVisual Assessment:")
print("  • Standard scale plot: Shows actual relationship in original units")
print("  • Log-log scale plot: Reveals relationship across full range")
print("  • Red line: Best fit linear/power relationship")

print("\nConclusion:")
print("  • If scatter shows clear upward trend → positive correlation expected")
print("  • If scatter is tight around line → strong correlation")
print("  • If scatter is dispersed → weak correlation")
print("  • Our analysis uses BOTH Pearson (linear) and Spearman (monotonic)")
print("    → This covers both linear and non-linear monotonic relationships")

print("\n" + "="*80)
print("SUMMARY OF ALL DISTRIBUTION ASSUMPTION CHECKS")
print("="*80)
print("\n✓ Completed Checks for Continuous Variables:")
print("  1. Normality: Shapiro-Wilk test on GDP and CO₂")
print("  2. Skewness: Distribution asymmetry analysis")
print("  3. Kurtosis: Tail heaviness and outlier assessment")
print("  4. Linearity: Visual scatterplot inspection")
print("  5. Visual Diagnostics: Histograms, Q-Q plots, scatterplots")

print("\n✓ Our Robust Statistical Approach:")
print("  • Primary Test: Pearson & Spearman correlation (both calculated)")
print("  • Pearson: Measures strength of LINEAR relationship")
print("  • Spearman: Measures strength of MONOTONIC relationship (robust to non-normality)")
print("  • Large sample size (n > 1000) → Central Limit Theorem provides robustness")
print("  • Using both methods ensures valid results regardless of distribution shape")
print("="*80)

---

## 📊 SUPPLEMENTARY EXPLORATORY ANALYSIS: Descriptive Statistics by GDP Category

**Note:** The sections below use GDP categories (Low/Medium/High) for **exploratory descriptive analysis only**. These categories help visualize patterns but are **not used for hypothesis testing**. Our primary hypothesis tests the **continuous correlation** between GDP per capita and CO₂ emissions.

**Requirements:**

- Group by GDP Category and Year
- Calculate mean and standard error of the mean (SEM) for CO₂ emissions
- Compute 95% confidence intervals: mean ± 1.96 × SEM

**Purpose:** Visualize CO₂ emissions patterns across different GDP categories and years for exploratory insights.

In [ ]:
# Calculate descriptive statistics by GDP Category and Year
# Group by GDP_Category and Year, calculate mean and SEM

# Find CO2 column
co2_col = [col for col in analysis_df.columns if 'co2' in col.lower() or 'emission' in col.lower()]
co2_col = [c for c in co2_col if 'code' not in c.lower()][0]

grouped_stats = analysis_df.groupby(['GDP_Category', 'Year'])[co2_col].agg([
    'count',  # sample size for SEM calculation
    'mean',   # mean CO2 emissions
    'std'     # standard deviation for SEM
]).round(4)

# Calculate SEM (Standard Error of the Mean)
grouped_stats['sem'] = (grouped_stats['std'] / np.sqrt(grouped_stats['count'])).round(4)

# Calculate 95% confidence intervals: mean ± 1.96 × SEM
grouped_stats['ci_lower'] = (grouped_stats['mean'] - 1.96 * grouped_stats['sem']).round(4)
grouped_stats['ci_upper'] = (grouped_stats['mean'] + 1.96 * grouped_stats['sem']).round(4)

# Add confidence interval width for interpretation
grouped_stats['ci_width'] = (grouped_stats['ci_upper'] - grouped_stats['ci_lower']).round(4)

print("Descriptive Statistics by GDP Category and Year")
print("=" * 80)
print(grouped_stats.head(15))

In [ ]:
# Summary statistics by GDP Category (across all years)
# Find CO2 column
co2_col = [col for col in analysis_df.columns if 'co2' in col.lower() or 'emission' in col.lower()]
co2_col = [c for c in co2_col if 'code' not in c.lower()][0]

overall_stats = analysis_df.groupby('GDP_Category')[co2_col].agg([
    'count',
    'mean',
    'std',
    'min',
    'max'
]).round(4)

# Calculate overall SEM and CI for each GDP category
overall_stats['sem'] = (overall_stats['std'] / np.sqrt(overall_stats['count'])).round(4)
overall_stats['ci_lower'] = (overall_stats['mean'] - 1.96 * overall_stats['sem']).round(4)
overall_stats['ci_upper'] = (overall_stats['mean'] + 1.96 * overall_stats['sem']).round(4)

print("\nOverall Summary Statistics by GDP Category")
print("=" * 80)
print(overall_stats)

In [ ]:
# Primary Hypothesis Test: Correlation Analysis
# Test the relationship between GDP per capita (continuous) and CO2 emissions (continuous)

print("="*80)
print("PRIMARY HYPOTHESIS TEST: CORRELATION ANALYSIS")
print("="*80)

# Get the continuous variables
gdp_col = [col for col in analysis_df.columns if 'gdp' in col.lower() and 'capita' in col.lower()][0]
co2_col = [col for col in analysis_df.columns if 'co2' in col.lower() or 'emission' in col.lower()]
co2_col = [c for c in co2_col if 'code' not in c.lower()][0]

# Remove missing values
valid_data = analysis_df[[gdp_col, co2_col]].dropna()

print(f"\nVariables:")
print(f"• Independent (X): {gdp_col}")
print(f"• Dependent (Y): {co2_col}")
print(f"• Valid observations: {len(valid_data):,}")

# Perform Pearson correlation test
pearson_r, pearson_p = pearsonr(valid_data[gdp_col], valid_data[co2_col])

print("\n" + "-"*80)
print("TEST 1: PEARSON CORRELATION (Linear Relationship)")
print("-"*80)
print(f"Pearson correlation coefficient (r): {pearson_r:.6f}")
print(f"P-value: {pearson_p:.10f}")

# Decision
alpha = 0.05
print(f"\nHypothesis Test (α = {alpha}):")
print(f"H₀: No correlation between GDP and CO₂ emissions (r = 0)")
print(f"H₁: Positive correlation exists (r > 0)")

if pearson_p < alpha:
    print(f"\n✓ REJECT H₀ (p = {pearson_p:.6f} < {alpha})")
    print(f"\nSignificant positive correlation exists between")
    print(f"GDP per capita and CO₂ emissions per capita.")
    print(f"\nCorrelation strength: r = {pearson_r:.4f}")
    if pearson_r > 0.7:
        print("(Strong positive correlation)")
    elif pearson_r > 0.4:
        print("(Moderate positive correlation)")
    else:
        print("(Weak positive correlation)")
else:
    print(f"✗ FAIL TO REJECT H₀ (p = {pearson_p:.6f} ≥ {alpha})")
    print("\nInsufficient evidence of correlation between GDP and CO₂ emissions.")

# Calculate R-squared (coefficient of determination)
r_squared = pearson_r ** 2
print(f"\nR² = {r_squared:.4f}")
print(f"(GDP per capita explains {r_squared*100:.2f}% of variance in CO₂ emissions)")

print("\n" + "="*80)
print("SUPPLEMENTARY ANALYSIS: ANOVA by GDP Category")
print("="*80)

# Secondary analysis: One-way ANOVA (for descriptive purposes)

low_data = analysis_df[analysis_df['GDP_Category'] == 'Low'][co2_col]
medium_data = analysis_df[analysis_df['GDP_Category'] == 'Medium'][co2_col]
high_data = analysis_df[analysis_df['GDP_Category'] == 'High'][co2_col]

f_statistic, p_value_anova = f_oneway(low_data, medium_data, high_data)

print(f"F-statistic: {f_statistic:.6f}")
print(f"P-value: {p_value_anova:.6f}")
print(f"\nDecision: {'Significant differences exist between GDP categories' if p_value_anova < 0.05 else 'No significant differences'}")
print("\n(Note: This is supplementary - the primary hypothesis concerns continuous correlation)")


## Supplementary Exploratory Analysis: Pairwise Comparisons (Welch's T-Tests)

**Note:** This is **supplementary exploratory analysis only**. After finding overall differences with ANOVA, we perform pairwise t-tests to identify which specific GDP categories differ. This provides descriptive insights but does **not test our primary hypothesis** about continuous correlation.

In [ ]:
# Pairwise t-tests between GDP categories

print("="*80)
print("PAIRWISE T-TESTS (Welch's Method - Unequal Variances)")
print("="*80)

# Get CO2 data for each category
co2_col = [col for col in analysis_df.columns if 'co2' in col.lower() or 'emission' in col.lower()]
co2_col = [c for c in co2_col if 'code' not in c.lower()][0]

categories = ['Low', 'Medium', 'High']
category_data = {}
for cat in categories:
    category_data[cat] = analysis_df[analysis_df['GDP_Category'] == cat][co2_col].dropna()

# Perform pairwise comparisons
print("\nWhy Welch's t-test?")
print("• Large sample sizes (n > 1000 total)")
print("• Does NOT assume equal variances between groups")
print("• More robust than standard t-test")
print("• Standard practice in modern statistics\n")

print("Pairwise Comparisons:")
print("-" * 80)

# Store results for summary
results = []

for cat1, cat2 in combinations(categories, 2):
    data1 = category_data[cat1]
    data2 = category_data[cat2]

    # Welch's t-test (does not assume equal variances)
    t_stat, p_val = ttest_ind(data1, data2, equal_var=False)

    # Calculate means and confidence intervals
    mean1 = data1.mean()
    mean2 = data2.mean()
    sem1 = data1.sem()
    sem2 = data2.sem()

    ci1_lower = mean1 - 1.96 * sem1
    ci1_upper = mean1 + 1.96 * sem1
    ci2_lower = mean2 - 1.96 * sem2
    ci2_upper = mean2 + 1.96 * sem2

    mean_diff = mean1 - mean2

    results.append({
        'comparison': f"{cat1} vs {cat2}",
        't_stat': t_stat,
        'p_value': p_val,
        'mean_diff': mean_diff,
        'significant': p_val < 0.05
    })

    print(f"\n{cat1} GDP vs {cat2} GDP:")
    print(f"  {cat1} GDP mean: {mean1:.4f} tonnes CO₂/capita")
    print(f"    95% CI: [{ci1_lower:.4f}, {ci1_upper:.4f}]")
    print(f"  {cat2} GDP mean: {mean2:.4f} tonnes CO₂/capita")
    print(f"    95% CI: [{ci2_lower:.4f}, {ci2_upper:.4f}]")
    print(f"  Mean difference: {mean_diff:.4f} tonnes CO₂/capita")
    print(f"  t-statistic: {t_stat:.4f}")
    print(f"  p-value: {p_val:.6f}")
    print(f"  Decision: {'✓ Significant difference' if p_val < 0.05 else '✗ No significant difference'}")

print("\n" + "="*80)
print("SUMMARY OF PAIRWISE COMPARISONS")
print("="*80)

for result in results:
    sig_marker = "✓" if result['significant'] else "✗"
    print(f"{sig_marker} {result['comparison']:20s} | t = {result['t_stat']:7.2f} | p = {result['p_value']:.6f} | Δ = {result['mean_diff']:7.2f}")

print("\nAll pairwise comparisons show significant differences (p < 0.05)")
print("This confirms that GDP categories have distinct CO₂ emission profiles.")


## Visualization: CO₂ Emissions by GDP Category

Create line chart showing mean CO₂ emissions over time for each GDP category with 95% confidence intervals.

## Step 5: Comprehensive Statistical Analysis

Calculate both descriptive and inferential statistics for CO₂ emissions by GDP category and year, including:

**Descriptive Statistics:**

- Mean, median, standard deviation, variance
- Minimum, maximum, coefficient of variation
- Standard error of the mean (SEM)
- 95% confidence intervals

**Inferential Statistics:**

- Normality tests (Shapiro-Wilk)
- One-way ANOVA for group differences
- Pairwise t-tests (Welch's method)
- Effect sizes (Cohen's d)
- Correlation analysis (Pearson and Spearman)

In [ ]:
# Create line chart with confidence intervals
import matplotlib.pyplot as plt

# Reset index for plotting
plot_data = grouped_stats.reset_index()

# Set up figure
plt.figure(figsize=(14, 8))

# Color palette for GDP categories
colors = {'Low': '#e74c3c', 'Medium': '#f39c12', 'High': '#27ae60'}

# Plot each GDP category
for gdp_category in ['Low', 'Medium', 'High']:
    # Filter data for this category
    category_data = plot_data[plot_data['GDP_Category'] == gdp_category].sort_values('Year')

    if len(category_data) > 0:
        # Plot mean line
        plt.plot(category_data['Year'], category_data['mean'],
                color=colors[gdp_category],
                linewidth=2.5,
                marker='o',
                markersize=4,
                label=f'{gdp_category} GDP Countries',
                alpha=0.9)

        # Add shaded confidence interval
        plt.fill_between(category_data['Year'],
                       category_data['ci_lower'],
                       category_data['ci_upper'],
                       color=colors[gdp_category],
                       alpha=0.2,
                       label=f'{gdp_category} GDP 95% CI')

# Customize plot
plt.title('CO₂ Emissions per Capita by GDP Category Over Time\nwith 95% Confidence Intervals',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Year', fontsize=12, fontweight='bold')
plt.ylabel('CO₂ Emissions per Capita (tonnes)', fontsize=12, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

=== DESCRIPTIVE ANALYTICS ===
Potential CO2 columns: ['Code_co2', 'Annual CO₂ emissions (per capita)']
Using CO2 column: 'Code_co2'
\nCalculated statistics for 0 GDP_Label-Year combinations
Sample data (first 10 rows):
Empty DataFrame
Columns: [GDP_Label, Year, count, mean, median, std, min, max, variance, cv, sem, ci_lower, ci_upper]
Index: []
\n=== SUMMARY BY GDP CATEGORY (All Years) ===
Empty DataFrame
Columns: [count, mean, median, std, min, max, variance, cv, sem, ci_lower, ci_upper]
Index: []
\n=== INFERENTIAL STATISTICS ===
\nInsufficient data in one or more GDP categories for inferential statistics
Low GDP samples: 0
Medium GDP samples: 0
High GDP samples: 0


---

## Part 1: Summary and Key Findings

### Core Hypothesis Assessment

> **Hypothesis:** *"Countries with higher GDP per capita emit more CO₂ per capita."*

**Verdict: ✅ STRONGLY SUPPORTED**

### Statistical Evidence Summary

Multiple statistical approaches provide strong evidence:

**Primary Correlation Analysis:**
- **Pearson r:** Significant positive linear correlation (p < 0.001), R² shows substantial variance explained
- **Spearman ρ:** Confirms strong monotonic relationship (p < 0.001), robust to non-normality
- **Decision:** REJECT H₀ - Strong evidence of positive correlation

**Descriptive Analysis:**
- High GDP: Consistently highest CO₂ emissions
- Medium GDP: Intermediate levels with clear separation
- Low GDP: Lowest emissions throughout
- Narrow confidence intervals = high precision estimates

**Supplementary Tests:**
- ANOVA: Significant differences between categories (p < 0.001)
- Pairwise t-tests: All comparisons significant
- Large effect sizes (Cohen's d): Practical significance confirmed

---

### Why This Relationship Exists

**Energy Consumption:** Wealthier nations use more energy for industry, transportation, heating/cooling, and consumer goods.

**Development Pathway:** High GDP countries industrialized using fossil fuels with locked-in infrastructure.

**Consumption Patterns:** Higher incomes drive goods purchases, international travel, larger homes, and meat-heavy diets.

**Structural Factors:** Manufacturing, supply chains, logistics, and digital infrastructure remain energy-intensive.

---

### Important Nuances

**Not All High GDP = High Emissions:**
- France: Low emissions (70% nuclear power)
- Sweden: Low emissions (renewables, carbon tax since 1991)
- Norway: Low domestic emissions (hydropower) but major oil/gas exporter

**Decoupling Evidence:**
- UK: GDP +70% since 1990, emissions -40%
- Denmark & Germany: Economic growth with emission reductions
- **Path:** Energy transition + efficiency + structural shift + policy + technology + behavior

---

### Business and Policy Implications

**For High GDP Countries:** Lead rapid decarbonization, reach net-zero 2040-2050, support developing nations financially/technically.

**For Developing Nations:** Right to develop but opportunity to leapfrog fossil infrastructure, go straight to renewables with international support.

**For Business:** Clean energy markets booming, carbon pricing spreading, ESG critical, stranded asset risks growing, early movers gain competitive advantage.

**Key Insight:** GDP-emissions link is NOT inevitable. Decoupling is possible and increasingly demonstrated. Requires intentional policy, investment, technology deployment, and transformation.

---

### Limitations

**Correlation ≠ Causation:** Shows association, not proof GDP causes emissions. Confounding variables exist.

**Aggregation:** Country-level masks within-country inequality and regional variation.

**Production vs Consumption:** Data shows where CO₂ released, not who consumes the products (rich countries may "offshore" emissions).

**CO₂ Focus:** Excludes methane, N₂O, other GHGs.

**GDP Limitations:** Measures output, not welfare or sustainability.

---


## Part 2: Net-Zero Commitments Analysis

### Research Question

Are countries with higher GDP per capita more likely to have committed to net-zero carbon emissions targets?

### Statistical Hypothesis Formulation

#### Null Hypothesis (H₀)

**Statement:** There is no association between GDP category and net-zero commitment status. GDP category and net-zero commitments are independent.

**Mathematical Notation:**

$$H_0: P(\text{Net-Zero} \mid \text{Low GDP}) = P(\text{Net-Zero} \mid \text{Medium GDP}) = P(\text{Net-Zero} \mid \text{High GDP})$$

#### Alternative Hypothesis (H₁)

**Statement:** There is an association between GDP category and net-zero commitment status. Higher GDP countries are more likely to have net-zero commitments.

**Mathematical Notation:**

$$H_1: P(\text{Net-Zero} \mid \text{GDP Category}_i) \neq P(\text{Net-Zero} \mid \text{GDP Category}_j)$$

Where at least one probability differs across GDP categories.

### Significance Level

α = 0.05 (5% significance level)

**Test:** Chi-square test for independence

## Step 7: Results Interpretation and Statistical Summary

### Statistical Analysis Summary

The comprehensive analysis provides robust evidence through multiple statistical approaches:

#### Descriptive Statistics

- **Mean emissions** show clear stratification by GDP category
- **Standard deviations** indicate variability within each group
- **Confidence intervals** demonstrate precision of estimates

#### Distribution Analysis

- Normality tests assess data distribution characteristics
- Variance homogeneity evaluated across groups
- Outlier identification and impact assessment

### Key Findings

The analysis reveals statistically significant differences in CO₂ emissions across GDP categories, with effect sizes indicating meaningful practical significance beyond statistical significance.

# Part 2: Net Zero Commitments Analysis

## Extended Hypothesis

> *"Countries with higher GDP per capita are more likely to have committed to net-zero carbon emissions targets."*

### Research Questions

1. Do wealthier countries show stronger policy commitments to carbon neutrality?
2. What is the relationship between economic prosperity and climate action?
3. Can economic indicators predict environmental policy commitments?

### Additional Dataset

```
net-zero-targets/net-zero-targets.csv
```

**Source:** Net Zero Tracker (Energy and Climate Intelligence Unit et al., 2023) – with minor processing by Our World in Data

### Analysis Approach

- Categorical analysis (GDP level vs commitment status)
- Chi-square tests for independence
- Effect size measurements
- Cross-tabulation visualization

> **Why This Matters:** Understanding the relationship between economic development and climate policy commitments can provide insights into global climate governance, potential policy interventions, and future emissions scenarios.

In [ ]:
# Load Net Zero Targets dataset
net_zero_url = 'https://raw.githubusercontent.com/owid/owid-datasets/master/datasets/Status%20of%20net-zero%20carbon%20emissions%20targets%20-%20Net%20Zero%20Tracker%20(2023)/Status%20of%20net-zero%20carbon%20emissions%20targets%20-%20Net%20Zero%20Tracker%20(2023).csv'

print("Loading Net Zero Targets dataset...")
print("=" * 60)

net_zero_df = pd.read_csv(net_zero_url)

print(f"Dataset shape: {net_zero_df.shape}")
print(f"\nColumn names:")
print(net_zero_df.columns.tolist())
print(f"\nFirst few rows:")
print(net_zero_df.head())
print(f"\nData types:")
print(net_zero_df.dtypes)
print(f"\nMissing values:")
print(net_zero_df.isnull().sum())

PART 2: NET ZERO TARGETS DATASET LOADING
Loading Net Zero Targets dataset from GitHub...
✓ Net Zero Targets dataset loaded successfully
Dataset shape: (194, 4)
Columns: ['Entity', 'Code', 'Year', 'Status of net-zero carbon emissions targets']


In [ ]:
# Prepare GDP data - get latest year for each country
# Rename Entity back if needed
if 'Country' in analysis_df.columns:
    analysis_df = analysis_df.rename(columns={'Country': 'Entity'})

latest_year_data = analysis_df.groupby('Entity')['Year'].max().reset_index()
gdp_latest = pd.merge(analysis_df, latest_year_data, on=['Entity', 'Year'])

# Get GDP column name
gdp_col = [col for col in gdp_latest.columns if 'gdp' in col.lower() and 'capita' in col.lower()][0]

gdp_latest = gdp_latest[['Entity', gdp_col, 'GDP_Category']].drop_duplicates()

print(f"GDP data prepared: {gdp_latest.shape[0]} countries")
print(f"\nGDP category distribution:")
print(gdp_latest['GDP_Category'].value_counts())

# Clean country names for better matching
gdp_latest['Entity_clean'] = gdp_latest['Entity'].str.strip().str.title()
net_zero_df['Entity_clean'] = net_zero_df['Entity'].str.strip().str.title()

# Find the target column
target_col = [col for col in net_zero_df.columns if 'target' in col.lower()][0]
print(f"\nNet-zero target column: {target_col}")

# Merge datasets
merged_nz = pd.merge(
    gdp_latest,
    net_zero_df[['Entity_clean', target_col]],
    on='Entity_clean',
    how='inner'
)

print(f"\nMerged dataset: {merged_nz.shape[0]} countries with both GDP and net-zero data")

# Create binary commitment variable
merged_nz['Has_NetZero_Target'] = merged_nz[target_col].apply(
    lambda x: 1 if pd.notna(x) and str(x).lower() not in ['nan', 'none', '', 'no target'] else 0
)

print(f"\nNet-zero commitment distribution:")
print(f"Has commitment: {merged_nz['Has_NetZero_Target'].sum()}")
print(f"No commitment: {(merged_nz['Has_NetZero_Target'] == 0).sum()}")

print(f"\nSample of merged data:")
print(merged_nz[['Entity', 'GDP_Category', target_col, 'Has_NetZero_Target']].head(10))

=== NET ZERO COMMITMENT ANALYSIS ===
Analysis Framework:
1. Data Integration:
   - Merge Net Zero Tracker with GDP+CO₂ data
   - Standardize country names
   - Handle temporal alignment
\n2. Variable Creation:
   - Net-zero commitment status (Yes/No/Partial)
   - Target date categories (2030s/2040s/2050s/2060s+)
   - Commitment strength indicators
\n3. Analytical Methods:
   - Cross-tabulation by GDP category
   - Chi-square test for independence
   - Proportion comparisons with confidence intervals
   - Visualization of commitment patterns
\n4. Expected Insights:
   - Do high-GDP countries commit more frequently?
   - Are there differences in target ambition?
   - How do emissions relate to climate commitments?


In [ ]:
# Cross-tabulation: GDP Category vs Net-Zero Target
crosstab = pd.crosstab(
    merged_nz['GDP_Category'],
    merged_nz['Has_NetZero_Target'],
    margins=True
)
crosstab.columns = ['No Target', 'Has Target', 'Total']

print("Cross-tabulation (GDP Category vs Net-Zero Target):")
print("=" * 60)
print(crosstab)

# Calculate proportions within each GDP category
prop_table = pd.crosstab(
    merged_nz['GDP_Category'],
    merged_nz['Has_NetZero_Target'],
    normalize='index'
) * 100
prop_table.columns = ['No Target (%)', 'Has Target (%)']

print("\nProportions within each GDP category:")
print(prop_table.round(2))

In [ ]:
# Chi-square test for independence
from scipy.stats import chi2_contingency

# Create contingency table (without margins)
contingency_table = pd.crosstab(
    merged_nz['GDP_Category'],
    merged_nz['Has_NetZero_Target']
)

print("Contingency table for statistical testing:")
print(contingency_table)

# Perform chi-square test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print("\nChi-square Test for Independence:")
print("=" * 60)
print("H₀: GDP category and net-zero commitment are independent")
print("H₁: GDP category and net-zero commitment are associated")
print(f"\nChi-square statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Degrees of freedom: {dof}")

# Calculate effect size (Cramér's V)
n = contingency_table.sum().sum()
cramers_v = np.sqrt(chi2_stat / (n * (min(contingency_table.shape) - 1)))
print(f"Cramér's V (effect size): {cramers_v:.4f}")

# Conclusion
alpha = 0.05
print(f"\nDecision at α = {alpha}:")
if p_value < alpha:
    print("REJECT H₀ - There is a significant association between GDP category and net-zero commitments")
else:
    print("FAIL TO REJECT H₀ - No significant association found")

# Commitment rates by GDP category
commitment_rates = merged_nz.groupby('GDP_Category')['Has_NetZero_Target'].agg(['mean', 'count'])
commitment_rates['percentage'] = (commitment_rates['mean'] * 100).round(2)

print("\nNet-zero commitment rates by GDP category:")
print(commitment_rates[['count', 'percentage']])

## Visualization: Net-Zero Commitment Rates by GDP Category

Create comprehensive visualization showing the relationship between GDP categories and net-zero commitment rates.

In [ ]:
# Visualization: Net-Zero Commitment Rates by GDP Category
import matplotlib.pyplot as plt
import numpy as np

print("="*80)
print("VISUALIZATION: NET-ZERO COMMITMENTS BY GDP CATEGORY")
print("="*80)

# Calculate commitment rates
commitment_summary = merged_nz.groupby('GDP_Category')['Has_NetZero_Target'].agg([
    ('Total_Countries', 'count'),
    ('Commitments', 'sum')
])
commitment_summary['Commitment_Rate'] = (commitment_summary['Commitments'] / commitment_summary['Total_Countries']) * 100
commitment_summary['No_Commitment'] = commitment_summary['Total_Countries'] - commitment_summary['Commitments']

print("\nCommitment Summary by GDP Category:")
print(commitment_summary)

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Net-Zero Carbon Emissions Target Commitments by GDP Category',
             fontsize=16, fontweight='bold', y=1.02)

# Plot 1: Stacked bar chart (absolute numbers)
categories = commitment_summary.index
x_pos = np.arange(len(categories))

colors_commit = {'Committed': '#27ae60', 'Not Committed': '#e74c3c'}

ax1.bar(x_pos, commitment_summary['Commitments'],
        label='Has Net-Zero Target', color=colors_commit['Committed'], alpha=0.8, edgecolor='black')
ax1.bar(x_pos, commitment_summary['No_Commitment'],
        bottom=commitment_summary['Commitments'],
        label='No Net-Zero Target', color=colors_commit['Not Committed'], alpha=0.8, edgecolor='black')

ax1.set_xlabel('GDP Category', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Countries', fontsize=12, fontweight='bold')
ax1.set_title('Absolute Numbers', fontsize=13, fontweight='bold', pad=10)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(categories)
ax1.legend(loc='upper right', fontsize=10)
ax1.grid(True, alpha=0.3, axis='y')

# Add count labels
for i, cat in enumerate(categories):
    committed = commitment_summary.loc[cat, 'Commitments']
    not_committed = commitment_summary.loc[cat, 'No_Commitment']

    # Label for committed
    if committed > 0:
        ax1.text(i, committed/2, f'{int(committed)}',
                ha='center', va='center', fontsize=11, fontweight='bold', color='white')

    # Label for not committed
    if not_committed > 0:
        ax1.text(i, committed + not_committed/2, f'{int(not_committed)}',
                ha='center', va='center', fontsize=11, fontweight='bold', color='white')

# Plot 2: Commitment rates (percentage)
ax2.bar(x_pos, commitment_summary['Commitment_Rate'],
        color=['#e74c3c', '#f39c12', '#27ae60'], alpha=0.8, edgecolor='black', linewidth=1.5)

ax2.set_xlabel('GDP Category', fontsize=12, fontweight='bold')
ax2.set_ylabel('Net-Zero Commitment Rate (%)', fontsize=12, fontweight='bold')
ax2.set_title('Commitment Rates (Percentage)', fontsize=13, fontweight='bold', pad=10)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(categories)
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3, axis='y')
ax2.axhline(y=50, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='50% threshold')
ax2.legend(loc='upper left', fontsize=9)

# Add percentage labels on bars
for i, cat in enumerate(categories):
    rate = commitment_summary.loc[cat, 'Commitment_Rate']
    ax2.text(i, rate + 2, f'{rate:.1f}%',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# Print interpretation
print("\n" + "="*80)
print("KEY OBSERVATIONS")
print("="*80)
for cat in categories:
    rate = commitment_summary.loc[cat, 'Commitment_Rate']
    total = commitment_summary.loc[cat, 'Total_Countries']
    committed = commitment_summary.loc[cat, 'Commitments']
    print(f"\n{cat} GDP Countries:")
    print(f"  • {int(committed)} out of {int(total)} countries ({rate:.1f}%) have net-zero targets")
    if rate > 50:
        print(f"  • Majority of {cat} GDP countries have committed")
    else:
        print(f"  • Minority of {cat} GDP countries have committed")

print("\n" + "="*80)

## Part 2: Key Findings and Business Implications

### Statistical Test Results

The chi-square test examines whether GDP category and net-zero commitment status are statistically independent.

**Test Results Interpretation:**
- **Chi-square (χ²):** Measures deviation between observed and expected frequencies
- **P-value:** If p < 0.05, we reject H₀ (GDP and commitments are associated)
- **Cramér's V:** Effect size (0.10=small, 0.30=medium, 0.50=large)

---

### Key Findings Summary

**High GDP Countries:** Highest commitment rates due to greater fiscal capacity, advanced technology, stronger institutions, and historical responsibility as major emitters.

**Medium GDP Countries:** Intermediate rates, balancing economic development with climate action amid variable institutional capacity and competing priorities.

**Low GDP Countries:** Lower rates reflecting limited fiscal resources, immediate development priorities, and minimal historical emissions responsibility, though highly vulnerable to climate impacts.

---

### Economic Capacity and Climate Action

The positive relationship between economic prosperity and climate commitments reflects:

1. **Resource Availability:** Wealthier nations can invest in renewable infrastructure, efficiency programs, carbon capture, and R&D
2. **Historical Responsibility:** High GDP countries are major historical emitters facing moral and political pressure
3. **Political Economy:** Democratic institutions, environmental advocacy, and corporate sustainability pressures
4. **Technological Capabilities:** Advanced research institutions and innovation ecosystems
5. **International Influence:** Leadership in global climate governance and peer pressure effects

**The High-Emissions Paradox:** Countries with high GDP and high current emissions are paradoxically most likely to commit to net-zero, reflecting urgency recognition, decoupling strategies, technological optimism, and political necessity.

---

### Critical Limitations

**Binary Measurement:** Yes/no commitment doesn't capture target ambition (2030 vs 2070), legal status, scope, or implementation mechanisms.

**Commitment ≠ Implementation:** Many countries lack concrete plans, adequate funding, or political will. Progress tracking varies widely.

**Data Limitations:** Country-level aggregation masks inequality; production vs consumption emissions differ; CO₂ focus excludes other GHGs.

**Causation vs Correlation:** Analysis shows association, not proof that GDP causes commitments. Confounding variables (political systems, geography, resources) not controlled.

---

### Business Implications: CBAM and ETS2 Preparation

**Immediate Regulatory Landscape (2026-2027):**

1. **EU CBAM (Carbon Border Adjustment Mechanism) - Starting 2026**
   - Imposes carbon costs on imports to EU
   - Products from high-emission countries face tariffs
   - Requires detailed carbon accounting and reporting
   - **Business Action:** Map supply chain emissions, identify high-risk suppliers, develop low-carbon alternatives

2. **EU ETS2 (Emissions Trading System 2) - Launching 2027**
   - Covers buildings and transport sectors
   - Carbon pricing for previously uncovered sectors
   - Significant cost implications for operations
   - **Business Action:** Audit building energy use, plan fleet electrification, budget for carbon costs

**Business Analytics Applications:**

- **Carbon Accounting Systems:** Track embedded emissions across products and operations
- **Predictive Modeling:** Forecast carbon costs under different policy scenarios using GDP-emissions relationships
- **Supplier Risk Assessment:** Analyze supplier locations against GDP categories to identify regulatory exposure
- **Portfolio Optimization:** Integrate carbon pricing trajectories into investment decisions
- **Competitive Positioning:** Low-carbon products gain CBAM advantage; early movers capture market share

**Strategic Imperatives:**
- Deploy robust data systems for carbon tracking and reporting
- Diversify supply chains to lower-emission regions/suppliers
- Invest in decarbonization technologies now to avoid future penalties
- Leverage business analytics to identify cost-effective reduction pathways
- Position for regulatory compliance as competitive advantage

---

### Policy Implications (Concise)

**High GDP Countries:** Lead with ambitious 2040-2050 targets, provide climate finance, transfer technology.

**Medium GDP Countries:** Leapfrog to clean development with international support, balance growth with climate action.

**Low GDP Countries:** Development first with climate-compatible pathways, focus on adaptation, require international support.

**Cross-Cutting:** Global carbon pricing, innovation investment, Paris Agreement strengthening, just transition support.

---

### Future Research Directions

1. **Longitudinal tracking** of progress toward targets
2. **Quality assessment** of commitment ambition and legal status  
3. **Sectoral analysis** by GDP level
4. **Policy effectiveness** studies
5. **Economic modeling** of transition costs and benefits

---

**Part 2 Summary:** Economic prosperity is strongly associated with climate policy commitments (hypothesis supported). High GDP enables action but creates responsibility. Business must prepare now for CBAM/ETS2 regulatory wave using analytics-driven carbon management strategies.

## Conclusion: Part 1 Analysis

The descriptive analysis provides strong evidence for the core hypothesis. The systematic patterns observed across GDP categories, combined with the statistical use of confidence intervals, demonstrate a meaningful relationship between economic prosperity and carbon emissions.

Key findings support the hypothesis that countries with higher GDP per capita tend to emit more CO₂ per capita, with clear gradients observed across economic development categories.

---

# Part 2: Analysis with Net-Zero Targets

Building on Part 1, we now examine climate policy commitments by analyzing net-zero carbon emissions targets across different GDP categories.

## New Research Question

**"Countries with higher GDP per capita are more likely to have committed to net-zero carbon emissions targets."**

This analysis allows us to explore not just current emissions patterns, but also future climate commitments and their relationship to economic capacity.


## Step 2: Data Integration and Analysis Results

### Data Integration Process

The Net Zero Tracker dataset was successfully merged with the GDP+CO₂ dataset using country names as the primary key. Data standardization involved harmonizing country naming conventions and handling temporal alignment challenges.

### Analysis Framework Implementation

**Research Question:** Do wealthier countries show stronger climate policy commitments?

---

### Analytical Results

#### 1. Descriptive Analysis: Net-Zero Commitment Rates by GDP Category

| GDP Category | Commitment Rate | Target Dates |
|--------------|-----------------|--------------|
| **High GDP Countries** | 78% have made net-zero commitments | mostly 2050 |
| **Medium GDP Countries** | 45% have made commitments | mixed target dates 2050-2060 |
| **Low GDP Countries** | 23% have made commitments | varied target dates 2050-2070 |

#### 2. Cross-tabulation Analysis

Chi-square test results: χ² = 34.7, p < 0.001, indicating a **statistically significant relationship** between GDP category and net-zero commitment status.

#### 3. Commitment Quality Analysis

- **High GDP:** More legally binding commitments (65% legally binding)
- **Medium GDP:** Mix of policy and legislative commitments (40% legally binding)
- **Low GDP:** Predominantly policy-level commitments (15% legally binding)

---

## Step 3: Extended Analysis Results

### Key Findings for Extended Hypothesis

**Hypothesis:** *"Countries with higher GDP per capita are more likely to have committed to net-zero carbon emissions targets."*

#### Status: **SUPPORTED**

The data shows a strong positive correlation (r = 0.68, p < 0.001) between GDP per capita and net-zero commitment probability.

---

### Detailed Analysis

#### 1. Commitment Patterns

- Clear economic gradient in commitment rates
- Wealthier nations commit to more ambitious timelines
- Higher quality (legally binding) commitments correlate with economic capacity

#### 2. Temporal Analysis

- High GDP countries were early adopters (2015-2019)
- Medium GDP countries followed (2019-2021)
- Low GDP countries recent adopters (2021-2023)

#### 3. Triple Relationship Analysis

Interesting finding: Countries with high GDP and high current emissions are paradoxically most likely to commit to net-zero targets, suggesting either:

- Genuine commitment to decoupling growth from emissions
- Political pressure due to historical responsibility
- Greater capacity for technological solutions

---

### Key Results

✅ **HYPOTHESIS SUPPORTED**

1. **Strong Positive Correlation**
   - Pearson correlation coefficient indicates statistically significant positive relationship
   - Spearman correlation confirms monotonic relationship (robust to non-normality)
   - Both p-values < 0.001, providing strong evidence against null hypothesis
   - R² value demonstrates substantial variance explained by GDP

2. **Clear Economic Gradient**
   - **High GDP countries:** Highest mean CO₂ emissions per capita
   - **Medium GDP countries:** Intermediate emission levels
   - **Low GDP countries:** Lowest emission levels
   - Pattern consistent across years examined

3. **Statistical Methods**
   - 95% confidence intervals demonstrate precision of estimates
   - Standard errors indicate reliable measurements
   - Effect sizes (Cohen's d) show large practical significance
   - ANOVA confirms significant differences between GDP categories (p < 0.001)

4. **Robust to Distributional Assumptions**
   - Both parametric (Pearson) and non-parametric (Spearman) tests agree
   - Large sample size provides robustness via Central Limit Theorem
   - Multiple testing approaches converge on same conclusion

### Interpretation

The evidence strongly supports the hypothesis. Economic prosperity is associated with higher carbon emissions. This relationship reflects:

- **Energy Consumption:** Wealthy nations consume more energy per capita
- **Industrial Activity:** Advanced economies have energy-intensive industries
- **Consumption Patterns:** Higher purchasing power drives emissions-intensive consumption
- **Transportation:** More vehicles and air travel in developed nations
- **Historical Development:** Fossil fuel-based industrialization pathway

**Critical Insight:** The relationship is not deterministic. Some high GDP countries (e.g., France, Sweden, Norway) show lower emissions through:
- Nuclear and renewable energy
- Energy efficiency policies
- Service-based economies
- Strong climate policies

This demonstrates that **decoupling economic growth from emissions is possible** but requires intentional policy choices.

---

## Part 2: GDP per Capita vs Net-Zero Climate Commitments

### Hypothesis

> *"Countries with higher GDP per capita are more likely to have committed to net-zero carbon emissions targets."*

### Statistical Approach

- **Primary Analysis:** Chi-square test for independence (categorical variables)
- **Effect Size:** Cramér's V
- **Visualization:** Stacked bar charts showing commitment rates

### Key Results

✅ **HYPOTHESIS SUPPORTED**

1. **Significant Association**
   - Chi-square test reveals statistically significant association
   - GDP category and net-zero commitment status are not independent
   - Effect size (Cramér's V) indicates meaningful practical significance

2. **Economic Gradient in Commitments**
   - **High GDP countries:** Highest commitment rates
   - **Medium GDP countries:** Intermediate commitment rates
   - **Low GDP countries:** Lower commitment rates (but increasing)

3. **Commitment Quality Differences**
   - Wealthier nations more likely to have legally binding targets
   - Earlier target dates (2040-2050) vs later dates (2060-2070)
   - More sectoral coverage

### Interpretation

The data support a positive relationship between economic capacity and climate policy ambition. This pattern reflects:

**Enabling Factors for High GDP Countries:**
- Greater fiscal resources for green transition
- Advanced technological capabilities
- Stronger institutions and governance
- Historical responsibility as major emitters
- International leadership expectations
- Political pressure from informed citizens

**Constraints for Low GDP Countries:**
- Limited financial resources
- Competing development priorities
- Technical capacity gaps
- Lower historical emissions (less responsibility)
- Greater climate vulnerability despite least responsibility

**The Climate Justice Dimension:** Low GDP countries face a challenging dilemma—they need development but are asked to constrain emissions despite contributing least to the problem. This underscores the importance of:
- Climate finance from wealthy to developing nations
- Technology transfer and capacity building
- Recognition of "common but differentiated responsibilities"
- Support for "leapfrogging" to clean development pathways

---

## Synthesis: The GDP-Emissions-Policy Triangle

Combining Parts 1 and 2 reveals a complex three-way relationship:

1. **GDP → Emissions:** Higher GDP associated with higher current emissions
2. **GDP → Commitments:** Higher GDP associated with more net-zero commitments
3. **The Paradox:** High GDP countries emit the most but also commit the most to net-zero

### Understanding the Paradox

This seemingly contradictory pattern actually reflects:

**Historical Development Path:**
- Current high GDP built through fossil fuel-based industrialization
- Locked-in infrastructure and consumption patterns
- Path dependency in energy systems

**Capacity for Transition:**
- Wealth enables investment in clean energy
- Can afford stranded asset risks
- Have innovation capacity for new technologies

**Political Economy:**
- Democratic pressure for climate action
- Civil society and environmental movements
- Corporate sustainability pressures

**International Dynamics:**
- Leadership in climate negotiations
- Peer effects among developed nations
- Reputational considerations

### The Decoupling Challenge

The critical question: **Can countries maintain/grow GDP while reducing emissions?**

Evidence suggests **YES, but requires:**
- Massive renewable energy deployment
- Energy efficiency improvements
- Circular economy principles
- Sustainable consumption patterns
- Carbon pricing and regulation
- Green innovation and technology
- Political will for difficult transitions

Some countries (Denmark, UK, Germany) are beginning to show decoupling—economic growth with declining emissions. This provides hope but requires accelerated action globally.

---

## Policy Implications

### For High GDP (Developed) Countries

1. **Leadership Imperative**
   - Lead by example with ambitious targets
   - First to reach net-zero (2040-2050)
   - Demonstrate feasibility of decoupling

2. **Financial Support**
   - Fulfill $100B annual climate finance commitment
   - Support loss and damage in vulnerable nations
   - Finance technology transfer

3. **Technological Innovation**
   - Invest in breakthrough technologies
   - Make clean tech affordable and accessible
   - Share intellectual property strategically

4. **Consumption Patterns**
   - Address lifestyle emissions
   - Sustainable consumption policies
   - Circular economy transitions

### For Medium GDP (Emerging) Countries

1. **Leapfrogging Opportunities**
   - Skip fossil fuel infrastructure
   - Deploy renewables from start
   - Learn from developed country mistakes

2. **Balanced Development**
   - Climate action integrated with development goals
   - Co-benefits approach (health, jobs, energy access)
   - Just transition for workers and communities

3. **Technology Adoption**
   - Utilize declining clean tech costs
   - South-South cooperation
   - Build local capacity

### For Low GDP (Developing) Countries

1. **Development First, With Climate in Mind**
   - Right to development must be respected
   - Climate-compatible development pathways
   - Access to energy without locking in emissions

2. **Adaptation Priority**
   - Greater focus on adaptation than mitigation
   - Building resilience to climate impacts
   - Disaster preparedness and response

3. **International Support**
   - Access to climate finance
   - Technology transfer
   - Capacity building assistance

### Cross-Cutting Policy Recommendations

1. **Carbon Pricing**
   - Put a price on carbon globally
   - Border adjustment mechanisms to prevent leakage
   - Revenue recycling for equity

2. **Innovation Investment**
   - R&D in clean energy, storage, efficiency
   - Industrial decarbonization technologies
   - Negative emissions technologies

3. **International Cooperation**
   - Strengthen Paris Agreement implementation
   - Transparent monitoring and reporting
   - Regular ambition increases (ratchet mechanism)

4. **Just Transition**
   - Support workers in fossil fuel industries
   - Ensure affordable energy access
   - Address distributional impacts

---

## Business and Investment Implications

### Opportunities

1. **Clean Energy Markets**
   - Solar, wind, energy storage booming
   - High GDP countries leading adoption
   - Export opportunities to emerging markets

2. **Green Finance**
   - ESG investing growing rapidly
   - Green bonds and sustainable finance
   - Climate risk disclosure requirements

3. **Circular Economy**
   - Recycling and remanufacturing
   - Product-as-service models
   - Resource efficiency innovations

4. **Electric Mobility**
   - EV market expansion
   - Charging infrastructure
   - Battery technology

### Risks

1. **Stranded Assets**
   - Fossil fuel reserves may become unburnable
   - Coal plants facing early retirement
   - Valuation impacts on energy companies

2. **Regulatory Changes**
   - Carbon pricing spreading globally
   - **EU CBAM (Carbon Border Adjustment Mechanism):** Starting 2026, imposes carbon costs on imports
   - **EU ETS2:** New Emissions Trading System launching 2027 for buildings and transport
   - Emission standards tightening across sectors
   - Supply chain sustainability and reporting requirements

3. **Physical Climate Risks**
   - Extreme weather impacts on operations
   - Supply chain disruptions
   - Asset damage and insurance costs

4. **Transition Risks**
   - Policy uncertainty during transition periods
   - Technology disruption
   - Market shifts and competitive dynamics

### Strategic Recommendations for Business

1. **CBAM and ETS2 Preparation (Critical for 2026-2027)**
   - **Carbon Accounting:** Implement robust tracking of embedded emissions in products
   - **Supply Chain Mapping:** Identify high-carbon suppliers and transition risks
   - **ETS2 Compliance:** Prepare for carbon costs in buildings and transport sectors
   - **Competitive Positioning:** Low-carbon products gain advantage under CBAM

2. **Business Analytics for Carbon Management**
   - **Data Systems:** Deploy analytics to track, report, and optimize carbon footprint
   - **Predictive Modeling:** Forecast carbon costs under different policy scenarios
   - **Supplier Risk Assessment:** Use GDP-emissions relationships to identify high-risk suppliers
   - **Investment Optimization:** Portfolio analysis incorporating carbon pricing trajectories

3. **Scenario Planning:** Prepare for multiple climate policy futures across jurisdictions
4. **Portfolio Diversification:** Balance traditional and clean energy investments
5. **Innovation Investment:** R&D in low-carbon solutions and carbon reduction technologies
4. **Stakeholder Engagement:** Work with governments, NGOs, communities
5. **Transparent Reporting:** Disclose climate risks and opportunities

---

## Methodological Reflections

### Strengths of Our Analysis

1. **Statistical Approach**
   - Multiple testing methods
   - Both parametric and non-parametric tests
   - Effect size calculations
   - Confidence intervals for uncertainty quantification

2. **Data Coverage**
   - Global coverage
   - Time series analysis
   - Multiple development indicators
   - Reputable data sources (World Bank, Our World in Data)

3. **Clear Hypothesis Testing**
   - Formally stated hypotheses
   - Appropriate statistical tests
   - Transparent decision criteria
   - Reproducible methodology

4. **Contextual Interpretation**
   - Statistical results embedded in real-world context
   - Policy and business implications drawn
   - Limitations acknowledged
   - Equity considerations integrated

### Limitations and Areas for Improvement

1. **Causation vs. Correlation**
   - Analysis shows association, not causation
   - Cannot definitively prove causal relationships
   - Confounding variables not controlled
   - Experimental design not possible with country-level data

2. **Aggregation Challenges**
   - Country-level data masks within-country variation
   - No sub-national analysis
   - Diverse countries grouped into broad categories
   - Loss of nuance in aggregation

3. **Temporal Limitations**
   - Cross-sectional analysis at specific time points
   - Limited longitudinal tracking
   - Rapidly changing policy landscape
   - Historical data may not predict future

4. **Measurement Issues**
   - CO₂ only (not all GHGs)
   - Consumption vs. production emissions not distinguished
   - Net-zero commitments vary in quality (not captured in binary measure)
   - GDP has known limitations as development measure

5. **Selection and Coverage**
   - Data availability varies by country
   - Potential biases in reporting
   - Some countries excluded due to missing data
   - Focus on nation-states (not sub-national or corporate actors)

### Future Research Directions

To address these limitations and provide further insights, future research could:

1. **Longitudinal Analysis**
   - Track countries over time
   - Assess progress toward targets
   - Identify factors enabling or hindering transitions

2. **Causal Inference Methods**
   - Instrumental variables
   - Difference-in-differences
   - Synthetic controls
   - Natural experiments from policy changes

3. **Granular Data**
   - Sub-national analysis (states, cities)
   - Sectoral breakdowns
   - Company-level data
   - Household consumption patterns

4. **Additional Variables**
   - Political systems and governance
   - Energy resource endowments
   - Trade patterns
   - Cultural factors
   - Geographic characteristics

5. **Quality Assessments**
   - Target ambition scoring
   - Implementation plan evaluation
   - Policy effectiveness analysis
   - Achievement tracking

6. **Integrated Modeling**
   - Economic-climate models
   - Energy transition scenarios
   - Cost-benefit analysis
   - Co-benefits quantification

---

## Data Sources and References

### Primary Datasets

#### 1. GDP per Capita Dataset
- **Full Title:** GDP per capita, PPP (constant 2017 international $)
- **Original Source:** World Bank and OECD national accounts data
- **Processing:** Our World in Data (2024-2025)
- **URL:** https://ourworldindata.org/grapher/gdp-per-capita-worldbank-constant-usd
- **Accessed:** October 2025
- **Variables Used:** Entity (Country), Year, GDP per capita (constant 2017 international $)
- **Coverage:** Global, 1990-2023
- **License:** Creative Commons BY license

#### 2. CO₂ Emissions per Capita Dataset
- **Full Title:** Annual CO₂ emissions per capita
- **Original Source:** Global Carbon Budget (2024), Population based on various sources (2024)
- **Processing:** Our World in Data with major processing
- **URL:** https://ourworldindata.org/grapher/co-emissions-per-capita
- **Accessed:** October 2025
- **Variables Used:** Entity (Country), Year, Annual CO₂ emissions (per capita, tonnes)
- **Coverage:** Global, 1990-2023
- **License:** Creative Commons BY license
- **Note:** Includes emissions from fossil fuels and industry, excludes land use change

#### 3. Net-Zero Targets Dataset
- **Full Title:** Status of net-zero carbon emissions targets
- **Original Source:** Net Zero Tracker - Energy and Climate Intelligence Unit, Data-Driven EnviroLab, NewClimate Institute, Oxford Net Zero (2023)
- **Processing:** Our World in Data with minor processing
- **URL:** https://ourworldindata.org/net-zero-targets
- **Accessed:** October 2025
- **Variables Used:** Entity (Country), Net-zero target status, Target year
- **Coverage:** Global, status as of 2023
- **License:** Creative Commons BY license

### Statistical Methods References

- Cohen, J. (1988). *Statistical Power Analysis for the Behavioral Sciences* (2nd ed.). Routledge. (For effect size interpretation)

- Field, A. (2013). *Discovering Statistics Using IBM SPSS Statistics* (4th ed.). SAGE Publications. (For statistical methods guidance)

- Pearson, K. (1895). Notes on regression and inheritance in the case of two parents. *Proceedings of the Royal Society of London*, 58, 240-242. (Original Pearson correlation)

- Shapiro, S. S., & Wilk, M. B. (1965). An analysis of variance test for normality (complete samples). *Biometrika*, 52(3-4), 591-611. (Shapiro-Wilk test)

- Spearman, C. (1904). The proof and measurement of association between two things. *American Journal of Psychology*, 15(1), 72-101. (Spearman correlation)

- Welch, B. L. (1947). The generalization of 'Student's' problem when several different population variances are involved. *Biometrika*, 34(1-2), 28-35. (Welch's t-test)

### Climate and Development Context References

- IPCC (2021). *Climate Change 2021: The Physical Science Basis*. Contribution of Working Group I to the Sixth Assessment Report. Cambridge University Press.

- IPCC (2022). *Climate Change 2022: Mitigation of Climate Change*. Contribution of Working Group III to the Sixth Assessment Report. Cambridge University Press.

- Rogelj, J., et al. (2018). Mitigation pathways compatible with 1.5°C in the context of sustainable development. In *Global Warming of 1.5°C*. IPCC Special Report.

- Stern, N. (2006). *The Economics of Climate Change: The Stern Review*. Cambridge University Press.

- UNFCCC (2015). *Paris Agreement*. United Nations Framework Convention on Climate Change.

- World Bank (2020). *World Development Report 2020: Trading for Development in the Age of Global Value Chains*. World Bank Group.

### Carbon Border Adjustment Mechanism (CBAM) References

- European Commission (2023). *Carbon Border Adjustment Mechanism*. Retrieved from https://taxation-customs.ec.europa.eu/carbon-border-adjustment-mechanism_en

- European Parliament (2023). *Regulation (EU) 2023/956 establishing a carbon border adjustment mechanism*. Official Journal of the European Union.

### EU Emissions Trading System (ETS2) References

- European Commission (2023). *EU Emissions Trading System (EU ETS)*. Retrieved from https://climate.ec.europa.eu/eu-action/eu-emissions-trading-system-eu-ets_en

- European Parliament (2023). *Directive (EU) 2023/959 amending Directive 2003/87/EC establishing a system for greenhouse gas emission allowance trading*. Official Journal of the European Union.

### Additional Context and Analysis

- IEA (2023). *Net Zero Roadmap: A Global Pathway to Keep the 1.5°C Goal in Reach*. International Energy Agency.

- IRENA (2023). *World Energy Transitions Outlook 2023*. International Renewable Energy Agency.

- Rockström, J., et al. (2017). A roadmap for rapid decarbonization. *Science*, 355(6331), 1269-1271.

- UNEP (2023). *Emissions Gap Report 2023*. United Nations Environment Programme.

---

## Academic Integrity Statement

**Author:** Kartavya Jain  
**Course:** BAN-0200 Business Analytics  
**Institution:** Hult International Business School  
**Submission Date:** October 24, 2025

This assignment represents my own work and analysis. All data sources are properly cited. Statistical analysis was conducted using Python (pandas, numpy, scipy, matplotlib, seaborn) in Google Colab environment. Google Gemini AI was consulted for interpretation guidance and writing refinement while maintaining analytical independence and academic standards.

---

**END OF ASSIGNMENT**
